In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from operator import add
import pyspark
spark=SparkSession.builder.appName("Demo").getOrCreate()

RDD = spark.sparkContext.textFile("/FileStore/tables/integer.txt")
RDD=RDD.map(lambda x:int(x)%2)
RDD=RDD.map(lambda word: (word, 1))
list=RDD.reduceByKey(add).collect()
print("odd:", list[1][1])
print("even:", list[0][1])

odd: 496
even: 514


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from operator import add
import pyspark
spark=SparkSession.builder.appName("Demo").getOrCreate()

RDD = spark.sparkContext.textFile("/FileStore/tables/salary.txt")
RDD=RDD.map(lambda line: (line.split(" ")[0],int(line.split(" ")[1])))
list=RDD.reduceByKey(add).collect()
print(list)

[('Sales', 3488491), ('Research', 3328284), ('Developer', 3221394), ('QA', 3360624), ('Marketing', 3158450)]


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from operator import add
import pyspark
spark=SparkSession.builder.appName("Demo").getOrCreate()

RDD = spark.sparkContext.textFile("/FileStore/tables/shakespeare_1.txt")
RDD=RDD.flatMap(lambda line: line.split(" "))
RDD=RDD.map(lambda word: (word, 1))
RDD=RDD.reduceByKey(lambda a, b: a + b)
for i in RDD.collect():
    if i[0] in ["Shakespeare", "why", "Lord", "Library", "GUTENBERG", "WILLIAM", "COLLEGE", "WORLD"]:
        print(i)

('Shakespeare', 22)
('GUTENBERG', 99)
('WILLIAM', 115)
('WORLD', 98)
('COLLEGE', 98)
('why', 91)
('Lord', 341)
('Library', 2)


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from operator import add
import pyspark
spark=SparkSession.builder.appName("Demo").getOrCreate()

RDD = spark.sparkContext.textFile("/FileStore/tables/shakespeare_1.txt")
RDD=RDD.flatMap(lambda line: line.split(" "))
RDD=RDD.map(lambda word: (word, 1))
RDD=RDD.reduceByKey(lambda a, b: a + b)
RDD=RDD.sortBy(lambda x: x[1],ascending=False).collect()
print("The top 10 words are:")
print(RDD[1:11])#Filter out empty line
print("The bottom 10 words are:")
print(RDD[-10:])

The top 10 words are:
[('the', 11397), ('and', 8777), ('I', 8556), ('of', 7873), ('to', 7421), ('a', 5672), ('my', 4913), ('in', 4600), ('you', 4060), ('And', 3547)]
The bottom 10 words are:
[('utterance,', 1), ('ruffle', 1), ('conspirators.', 1), ('loves?', 1), ('seventy-five', 1), ('drachmas.', 1), ('orchards,', 1), ('Tiber;', 1), ('Mischief,', 1), ('Octavius.', 1)]


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from operator import add
import pyspark
spark=SparkSession.builder.appName("Demo").getOrCreate()

RDD = spark.sparkContext.textFile("/FileStore/tables/movies.csv")
RDD=RDD.filter(lambda x:x!="movieId,rating,userId")
RDD=RDD.map(lambda line: (line.split(",")[0],int(line.split(",")[1]),line.split(",")[2]))

#count movies
movie=RDD.map(lambda word: (word[0], 1))
movie=movie.reduceByKey(lambda a, b: a + b)
#count total reviews
movie_r=RDD.map(lambda word: (word[0], word[1]))
movie_r=movie_r.reduceByKey(lambda a, b: a + b)
movie=movie.join(movie_r)
#compute avg rating
movie=movie.map(lambda word: (word[0], word[1][1]/word[1][0]))
movie=movie.sortBy(lambda x: x[1],ascending=False).collect()
print("The top 20 rated movies are:")
print(movie[0:20])

#count users
users=RDD.map(lambda word: (word[2], 1))
users=users.reduceByKey(lambda a, b: a + b)
#count total reviews
users_r=RDD.map(lambda word: (word[2], word[1]))
users_r=users_r.reduceByKey(lambda a, b: a + b)
users=users.join(users_r)
#compute avg rating
users=users.map(lambda word: (word[0], word[1][1]/word[1][0]))
users=users.sortBy(lambda x: x[1],ascending=False).collect()
print("The top 15 rateing users are:")
print(users[0:15])

The top 20 rated movies are:
[('32', 2.9166666666666665), ('90', 2.8125), ('30', 2.5), ('94', 2.473684210526316), ('23', 2.466666666666667), ('49', 2.4375), ('29', 2.4), ('18', 2.4), ('52', 2.357142857142857), ('53', 2.25), ('62', 2.25), ('92', 2.2142857142857144), ('46', 2.2), ('68', 2.1578947368421053), ('87', 2.1333333333333333), ('2', 2.1052631578947367), ('69', 2.076923076923077), ('27', 2.066666666666667), ('88', 2.0555555555555554), ('22', 2.05)]
The top 15 rateing users are:
[('11', 2.2857142857142856), ('26', 2.204081632653061), ('22', 2.1607142857142856), ('23', 2.1346153846153846), ('2', 2.0652173913043477), ('17', 1.9565217391304348), ('8', 1.8979591836734695), ('24', 1.8846153846153846), ('12', 1.8545454545454545), ('3', 1.8333333333333333), ('29', 1.826086956521739), ('28', 1.82), ('9', 1.7924528301886793), ('14', 1.7894736842105263), ('16', 1.7777777777777777)]


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator,TrainValidationSplit
spark=SparkSession.builder.appName("Demo").getOrCreate()

RDD = spark.sparkContext.textFile("/FileStore/tables/movies.csv").filter(lambda x:x!="movieId,rating,userId")
df=RDD.map(lambda line: (int(line.split(",")[0]),int(line.split(",")[1]),int(line.split(",")[2]))).toDF()
train, test = df.randomSplit(weights=[0.7, 0.3],seed=2)
als=ALS(userCol="_3",itemCol="_1",ratingCol="_2",coldStartStrategy="drop")

parameters=ParamGridBuilder().build()
eval = RegressionEvaluator(metricName= "rmse",labelCol= "_2",predictionCol= "prediction")

#Build train validation split
trainvs = TrainValidationSplit(estimator=als,estimatorParamMaps=parameters,evaluator=eval)
#Build cross validator
cv = CrossValidator(estimator =als,estimatorParamMaps =parameters,evaluator =eval,numFolds=3)

#Fit model to training data
model = trainvs.fit(train)
predictions = model.transform(test)

rmse = RegressionEvaluator(metricName= "rmse",labelCol= "_2",predictionCol= "prediction").evaluate(predictions)
print("RMSE=",rmse)
mse = RegressionEvaluator(metricName= "mse",labelCol= "_2",predictionCol= "prediction").evaluate(predictions)
print("MSE=",mse)
mae = RegressionEvaluator(metricName= "mae",labelCol= "_2",predictionCol= "prediction").evaluate(predictions)
print("MAE=",mae)

RMSE= 1.1814695827849822
MSE= 1.3958703750461199
MAE= 0.7863305180672572


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator,TrainValidationSplit
spark=SparkSession.builder.appName("Demo").getOrCreate()

RDD = spark.sparkContext.textFile("/FileStore/tables/movies.csv").filter(lambda x:x!="movieId,rating,userId")
df=RDD.map(lambda line: (int(line.split(",")[0]),int(line.split(",")[1]),int(line.split(",")[2]))).toDF()
train, test = df.randomSplit(weights=[0.7, 0.3],seed=2)
als=ALS(userCol="_3",itemCol="_1",ratingCol="_2",coldStartStrategy="drop")

parameters=ParamGridBuilder()\
.addGrid(als.rank,[16])\
.addGrid(als.maxIter, [10, 20])\
.addGrid(als.regParam, [0.05, 0.1])\
.addGrid(als.numItemBlocks, [5, 10])\
.addGrid(als.numUserBlocks, [5, 10]).build()

eval = RegressionEvaluator(metricName= "rmse",labelCol= "_2",predictionCol= "prediction")

#Build train validation split
trainvs = TrainValidationSplit(estimator=als,estimatorParamMaps=parameters,evaluator=eval)
#Build cross validator
cv = CrossValidator(estimator =als,estimatorParamMaps =parameters,evaluator =eval,numFolds=3)

#Fit model to training data
model = trainvs.fit(train)
predictions = model.transform(test)
#print(model.bestModel.rank)

rmse = RegressionEvaluator(metricName= "rmse",labelCol= "_2",predictionCol= "prediction").evaluate(predictions)
print("RMSE=",rmse)
mse = RegressionEvaluator(metricName= "mse",labelCol= "_2",predictionCol= "prediction").evaluate(predictions)
print("MSE=",mse)
mae = RegressionEvaluator(metricName= "mae",labelCol= "_2",predictionCol= "prediction").evaluate(predictions)
print("MAE=",mae)

RMSE= 1.2104203541555707
MSE= 1.4651174337540973
MAE= 0.7975462714062076


In [0]:
print(model.bestModel.rank)

16


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator,TrainValidationSplit
spark=SparkSession.builder.appName("Demo").getOrCreate()

df10 = [(i,-1, 10) for i in range(1, 101)]
df10 = spark.createDataFrame(df10, schema=["_1", "_2","_3"])
rec10=model.transform(df10)
print("the top 15 movie recommendations for userId 10 is:")
rec10.sort("prediction",ascending=False).show(15)

df14 = [(i,-1, 14) for i in range(1, 101)]
df14 = spark.createDataFrame(df14, schema=["_1", "_2","_3"])
rec14=model.transform(df14)
print("the top 15 movie recommendations for userId 14 is:")
rec14.sort("prediction",ascending=False).show(15)

the top 15 movie recommendations for userId 10 is:
+---+---+---+----------+
| _1| _2| _3|prediction|
+---+---+---+----------+
|  2| -1| 10|  3.150111|
| 42| -1| 10| 2.6256824|
| 53| -1| 10|  2.383279|
| 74| -1| 10| 1.8984079|
| 47| -1| 10| 1.8643752|
| 16| -1| 10| 1.8482214|
| 29| -1| 10| 1.7579231|
| 13| -1| 10| 1.7041345|
| 95| -1| 10| 1.6817774|
| 52| -1| 10| 1.6714218|
| 67| -1| 10| 1.6698463|
| 49| -1| 10| 1.6678468|
| 62| -1| 10| 1.6515498|
| 85| -1| 10| 1.6494863|
|  7| -1| 10| 1.5484833|
+---+---+---+----------+
only showing top 15 rows

the top 15 movie recommendations for userId 14 is:
+---+---+---+----------+
| _1| _2| _3|prediction|
+---+---+---+----------+
| 29| -1| 14| 4.8037715|
| 52| -1| 14| 4.5231886|
| 76| -1| 14| 4.2602873|
| 63| -1| 14|  4.132693|
| 62| -1| 14| 3.7405062|
| 72| -1| 14| 3.2885995|
| 70| -1| 14| 3.0416458|
| 95| -1| 14| 2.9015746|
| 31| -1| 14| 2.8322818|
| 47| -1| 14|  2.831221|
| 58| -1| 14| 2.7945845|
| 69| -1| 14| 2.7755332|
|  2| -1| 14|  2.74992